# Vehicle Sales Price Predictions - Part 1 of 3

## BEFORE WE START

This dataset can be downloaded to your PC from 
* [the following Google Drive link](https://drive.google.com/file/d/1GoXwFQCbKE0o32VE44G1NhpsvfPtX2d0/view?usp=drive_link)
* [this dowload location](https://repo.hops.works/dev/jdowling/car_prices.csv)

The original dataset can be found on [this Kaggle repository](https://www.kaggle.com/datasets/syedanwarafridi/vehicle-sales-data/data).

In order to make a *machine learning system* from this dataset, we have structured the service into 3 pipelines:
1. feature engineering pipeline notebook
2. training pipeline notebook
3. inferencing pipeline notebook

This notebook will start at the first step, ie. the feature engineering pipeline.

## FEATURE ENGINEERING PIPELINE

This is the first step of building our machine learning system. 

### 1. IMPORTING THE DATASET 
We have a dataset available at the above locations. How can we import this into a python variable.

In [ ]:
# Import the necessary libraries
import pandas as pd

#  Path to the source CSV file
file_path = "https://repo.hops.works/dev/jdowling/car_prices.csv"

# Read the source file and store in variable "data"
data = pd.read_csv(file_path)

### 2. VISUALISATION OF THE DATASET 
#### I want to visualise the file. How can I see the names of the columns with the 5 first values?

In [ ]:
# Show the name of the columns
print("Name of the columns:")
print(data.columns)

# Show the first 5 values
print("First 5 values:")
print(data.head())

#### How can I see how much data and how many variables (with their names and types) are available in the dataset?

In [ ]:
# Show info about the dataset
print(data.info())


#### Show the possible values of the numeric values in the dataset.

In [ ]:
# Select non-numeric columns
non_numeric_columns = data.select_dtypes(exclude=['float', 'int']).columns

# Afficher les valeurs uniques dans ces colonnes
for column in non_numeric_columns:
    print(column, data[column].unique())

#### How can we see if there are missing data or null values, and in which columns they reside, in the dataset?

In [ ]:
# Check if there are missing values or null-values in every column of the dataset
missing_values_per_column = data.isnull().sum()

# Display the columns with missing or null values
columns_with_missing_values = missing_values_per_column[missing_values_per_column > 0]
print("Columns with missing or null values:")
print(columns_with_missing_values)

#### Print a distribution chart for the values of the column "sellingprice"

In [ ]:
import matplotlib.pyplot as plt

# Parameters of the chart
plt.figure(figsize=(10, 6))  # Defines the size of the figure

# Trace the histogram
plt.hist(data['sellingprice'], bins=20, color='red', edgecolor='black')  # Use 20 bins (bars)

# Titles and labels
plt.title('Distribution of the values in the column "sellingprice"', fontsize=14)
plt.xlabel('Selling Price', fontsize=12)
plt.ylabel('Fréquence', fontsize=12)

# Show the graphic
plt.grid(True)  # Add a grid
plt.show()

#### What are the possible values of the column "year" ?

In [ ]:
# Find the unique values in the "year" column
values_year = data['year'].unique()

#  Show the unique values
print("Unique values in the 'year' column:")
print(sorted(values_year))

#### What are the possible values in the "make" column?

In [ ]:
# Find the unique values in the "make" column
values_make = data['make'].unique()

# Show the unique values
print("Show the unique values in the 'make' column :")
print(values_make)

#### What are the possible values in the "trim" column?

In [ ]:
# Find the unique values in the "trim" column
values_trim = data['trim'].unique()

# Show the unique values
print("Unique values in the 'trim' column:")
print(list(valeurs_trim))

### 3. CLEANING OF THE DATA

The dataet has the variables `vin`, `state`, `mmr` in it. We will remove these from the dataset.

In [ ]:
# Remove specific columns of the dataset
df = data.drop(['vin', 'state', 'mmr'], axis=1)

# Check if the columns have been removed
if all(col not in df.columns for col in ['vin', 'state', 'mmr']):
    print("The columns 'vin', 'state', 'mmr', 'seller' have been succesfully removed.")
else:
    print("The columns were not deleted correctly. Make sure the column names are correct.")

Remove all the missing data or `null` values from the dataset.

In [ ]:
# Number of lines before deletion
num_rows_before = df.shape[0]

# Delete all rows with missing or zero data
df1 = df.dropna()

# Number of lines after deletion
num_rows_after = df1.shape[0]

# Number of lines deleted
num_rows_deleted = num_rows_before - num_rows_after

print(f"Number of lines deleted: {num_rows_deleted}")

In [ ]:
# Additional changes to the dataframe wrt the "saledate" column: this will be necessary for our import into Hopsworks

from datetime import datetime, timezone

def convert_to_datetime_utc(date_str):
    try:
        # Remove the timezone abbreviation
        date_str = date_str.split(' (')[0]
        # Convert to datetime with timezone information
        dt_with_tz = datetime.strptime(date_str, '%a %b %d %Y %H:%M:%S GMT%z')
        # Convert to UTC and then remove timezone info
        dt_utc = dt_with_tz.astimezone(timezone.utc).replace(tzinfo=None)
        return dt_utc
    except Exception as e:
        print(f"Error converting date: {date_str} - {e}")
        return pd.NaT

# Apply the conversion function to the date_column
df1['saledate'] = df1['saledate'].apply(lambda x: convert_to_datetime_utc(x) if pd.notna(x) else pd.NaT)
df1

#### What is the average in the "sellingprice" column, what is the smallest and largest value?

In [ ]:
# Calculation of the average in the "sellingprice" column
avg_sellingprice = df1['sellingprice'].mean()
print("Average of the column 'sellingprice' :", avg_sellingprice)

# Smallest value in the "sellingprice" column
min_sellingprice = df1['sellingprice'].min()
print("Smallest value of the column 'sellingprice' :", min_sellingprice)

# Largest value in the "sellingprice" column
max_sellingprice = df1['sellingprice'].max()
print("Largest value in the column 'sellingprice' :", max_sellingprice)

#### Clears all data with a value less than or equal to 1000 or greater than 50000 in the "sellingprice" column. Also tell me how much data was deleted.

In [ ]:
# Count the number of data before deletion
lines_before = len(df1)

# Clear data with value smaller than 1000 or greater than 50000 in "sellingprice" column
df2 = df1[(df1['sellingprice'] > 1000) & (df1['sellingprice'] < 50000)]

# Count the number of data after deletion
lines_after = len(df2)

# Calculate the number of erased data
lines_removed = lines_before - lines_after

# Show number of erased data
print("Number of removed lines :", lines_removed)


#### The dataset has an "interior" column and a "color" column. Are there any missing, null, equal to "—" or numeric values ​​in this column? if yes, delete this data from the dataset and tell me how much data was deleted

In [ ]:
# Count the number of data before deletion
lines_before = len(df2)

# Clear data with missing, zero, or "—" values ​​in the "interior" and "color" columns
df3 = df2[(df2['interior'] != '—') & (df2['color'] != '—') & (df2['interior'].notnull()) & (df2['color'].notnull())]

# Count the number of data after deletion
lines_after = len(df3)

# Calculate the number of erased data
lines_removed = lines_before - lines_after

# Show number of removed lines
print("Number of removed lines:", lines_removed)

### 4. Insert the dataset into a Hopsworks Feature Group

In [ ]:
# Install the Hopsworks client library in the runtime. This will be used to connect to Hopsworks.
## Note that in Colab, this will throw an error but the system will still work.
!pip install --quiet hopsworks

In [ ]:
# data.to_csv('dataset_cleaned.csv', index=False)
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

In [ ]:
# Create a feature group in Hopsworks based on the above dataframe

fg = fs.get_or_create_feature_group(name="car_prices",
                                    version=2,
                                    description="The price of cars",
                                    primary_key=["seller", "saledate"],
                                    event_time="saledate"
                                    )
fg.insert(df3)

This completes the Feature Engineering pipeline of our demo project. We can now proceed to the Training Pipeline of the project.